In [9]:
import numpy as np
from collections import Counter

def calculate_conditionals(o, f):
    unique_forecasts = np.unique(f)
    o_cond_k = []
    f_cond_k = []
    p_k = []

    for k in unique_forecasts:
        indices = np.where(f == k)
        sub_o = o[indices]
        sub_f = f[indices]

        o_counter = Counter(sub_o)
        f_counter = Counter(sub_f)

        total_o = sum(o_counter.values())
        total_f = sum(f_counter.values())

        o_prob = np.array([o_counter[i]/total_o if i in o_counter else 0 for i in range(len(np.unique(o)))])
        f_prob = np.array([f_counter[i]/total_f if i in f_counter else 0 for i in range(len(np.unique(f)))])

        o_cond_k.append(o_prob)
        f_cond_k.append(f_prob)
        p_k.append(len(sub_o) / len(o))

    return np.array(o_cond_k), np.array(f_cond_k), np.array(p_k)

def DKL(o, f):
    return np.sum(o * np.log(o / (f + 1e-15) + 1e-15))

def UNC(o):
    return -np.sum(o * np.log(o + 1e-15))

def REL(o_cond_k, f_cond_k, p_k):
    rel = 0.0
    for k in range(len(p_k)):
        rel += p_k[k] * np.sum(o_cond_k[k, :] * np.log(o_cond_k[k, :] / (f_cond_k[k, :] + 1e-15) + 1e-15))
    return rel

def RES(o_cond_k, o, p_k):
    res = 0.0
    for k in range(len(p_k)):
        res += p_k[k] * np.sum(o_cond_k[k, :] * np.log(o_cond_k[k, :] / (o + 1e-15) + 1e-15))
    return res

# Example time series
o = np.array([0, 1, 0, 0, 0, 0, 1, 0])  # Observations
f = np.array([0.1, 0.5, 0.1, 0.2, 0.3, 0.4, 0.7, 0.6])  # Forecasts

# Calculate marginal probabilities
o_prob = np.array([Counter(o)[i]/len(o) for i in np.unique(o)])
f_prob = np.array([Counter(f)[i]/len(f) for i in np.unique(f)])

# Calculate conditional probabilities
o_cond_k, f_cond_k, p_k = calculate_conditionals(o, f)

# Compute components
unc = UNC(o_prob)
rel = REL(o_cond_k, f_cond_k, p_k)
res = RES(o_cond_k, o_prob, p_k)

# Compute DKL
dkl = unc - res + rel

print("DKL:", dkl, "\nUNC:", unc, "\nREL:", rel, "\nRES:", res)


ValueError: operands could not be broadcast together with shapes (2,) (7,) 